# Importando bibliotecas

In [414]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#modelling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


## Dataset

In [415]:
#importing the csv archive
df = pd.read_csv("high_diamond_ranked_10min.csv")
df.head()

df.info

<bound method DataFrame.info of           gameId  blueWins  blueWardsPlaced  blueWardsDestroyed  \
0     4519157822         0               28                   2   
1     4523371949         0               12                   1   
2     4521474530         0               15                   0   
3     4524384067         0               43                   1   
4     4436033771         0               75                   4   
...          ...       ...              ...                 ...   
9874  4527873286         1               17                   2   
9875  4527797466         1               54                   0   
9876  4527713716         0               23                   1   
9877  4527628313         0               14                   4   
9878  4523772935         1               18                   0   

      blueFirstBlood  blueKills  blueDeaths  blueAssists  blueEliteMonsters  \
0                  1          9           6           11                  0   
1    

## Analisando e limpando o Dataset

### Separando dados do *blue side* e do *red side*

In [416]:
#funcao p/ separar dados do red_side e do blue_side
def splitting_sides(side, side_data):
    for col in df:
        if side in col:
            side_data.append(col)
    
    return side_data

### *Blue Side*

In [417]:
blue_side = []
blue_side = splitting_sides("blue", blue_side)

blue_data = df[blue_side]

blue_data.head()

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,blueTowersDestroyed,blueTotalGold,blueAvgLevel,blueTotalExperience,blueTotalMinionsKilled,blueTotalJungleMinionsKilled,blueGoldDiff,blueExperienceDiff,blueCSPerMin,blueGoldPerMin
0,0,28,2,1,9,6,11,0,0,0,0,17210,6.6,17039,195,36,643,-8,19.5,1721.0
1,0,12,1,0,5,5,5,0,0,0,0,14712,6.6,16265,174,43,-2908,-1173,17.4,1471.2
2,0,15,0,0,7,11,4,1,1,0,0,16113,6.4,16221,186,46,-1172,-1033,18.6,1611.3
3,0,43,1,0,4,5,5,1,0,1,0,15157,7.0,17954,201,55,-1321,-7,20.1,1515.7
4,0,75,4,0,6,6,6,0,0,0,0,16400,7.0,18543,210,57,-1004,230,21.0,1640.0


### *Red Side* 

In [418]:
red_side = []
red_side = splitting_sides("red", red_side)

red_data = df[red_side]

red_data.head()

,redWardsPlaced,redWardsDestroyed,redFirstBlood,redKills,redDeaths,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,15,6,0,6,9,8,0,0,0,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,12,1,1,5,5,2,2,1,1,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,15,3,1,11,7,14,0,0,0,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,15,2,1,5,4,10,0,0,0,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,17,2,1,6,6,7,1,1,0,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


In [419]:
#adding a redWins column 

red_data["redWins"] = df["blueWins"].map({1:0, 0:1})
red_data.head() 

C:\Users\joaov\AppData\Local\Temp\ipykernel_7640\1815178656.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_data["redWins"] = df["blueWins"].map({1:0, 0:1})


,redWardsPlaced,redWardsDestroyed,redFirstBlood,redKills,redDeaths,redAssists,redEliteMonsters,redDragons,redHeralds,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin,redWins
0,15,6,0,6,9,8,0,0,0,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7,1
1,12,1,1,5,5,2,2,1,1,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0,1
2,15,3,1,11,7,14,0,0,0,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5,1
3,15,2,1,5,4,10,0,0,0,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8,1
4,17,2,1,6,6,7,1,1,0,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4,1


# Separando dados de teste e treinamento

In [420]:
#para o time vermelho
X = red_data.drop('redWins', axis=1).values

y = red_data['redWins'].values
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(X, y, test_size=0.30)


In [421]:
#para o time azul
X = blue_data.drop('blueWins', axis=1).values

y = blue_data['blueWins'].values
X_train_blue, X_test_blue, y_train_blue, y_test_blue = train_test_split(X, y, test_size=0.30)

# Modelando os dados usando *Random Forest* 


In [422]:
#Treinando vermelho
rdf = RandomForestClassifier(n_estimators=40, max_depth= 10)
rdf.fit(X_train_red, y_train_red)

RandomForestClassifier(max_depth=10, n_estimators=40)

In [423]:
#treinando azul
rdf = RandomForestClassifier(n_estimators=40, max_depth= 10)
rdf.fit(X_train_blue, y_train_blue)

RandomForestClassifier(max_depth=10, n_estimators=40)

## Testanto o modelo

In [424]:
#testanto Vermelho
y_pred = rdf.predict(X_test_red)

accuracy = accuracy_score(y_test_red, y_pred)
precision = precision_score(y_test_red, y_pred)
recall = recall_score(y_test_red, y_pred)

print("Accuracy:", "{:.5f}".format(accuracy))
print("Precision:", "{:.5f}".format(precision))
print("Recall:", "{:.5f}".format(recall))

Accuracy: 0.71829
Precision: 0.72641
Recall: 0.71238


In [425]:
#testando azul
y_pred = rdf.predict(X_test_blue)

accuracy = accuracy_score(y_test_blue, y_pred)
precision = precision_score(y_test_blue, y_pred)
recall = recall_score(y_test_blue, y_pred)

print("Accuracy:", "{:.5f}".format(accuracy))
print("Precision:", "{:.5f}".format(precision))
print("Recall:", "{:.5f}".format(recall))

Accuracy: 0.72503
Precision: 0.73632
Recall: 0.72423


## Treinando com o database completo

### Separando conjunto de teste e treinamento


In [426]:
X = df.drop('blueWins', axis=1).values

y = df['blueWins'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

### Treinando o modelo


In [427]:
rdf = RandomForestClassifier(n_estimators=50)
rdf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50)

<H2> Testando o modelo

In [428]:
y_pred = rdf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", "{:0.5f}".format(accuracy))
print("Precision:", "{:0.5f}".format(precision))
print("Recall:", "{:0.5f}".format(recall))

Accuracy: 0.73077
Precision: 0.73679
Recall: 0.71891


# Análise dos K Neighbors (previsão de vitória do *blue side*)

Dentro do League of Legends, um importante fator que leva a vitória de uma equipe é o KDA médio de uma equipe. O KDA consite na quantidade de *Kills*, *Deaths* e *Assist* de uma equipe. Além disso, a quantidade de ouro total resgatada pelos jogadores também é um fator importante, pois o dinheiro adquirido durante uma partida será utilizado na compra de itens para deixar a sua equipe mais fortalecida. 

In [437]:
X = blue_data.loc[:, ['blueKills', 'blueDeaths','blueAssists', 'blueTotalGold']].values #independent variables considering the KDA and total gold

y = blue_data.iloc[:, 0].values #dependent variables aka blueWins datas

In [438]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


classifier = KNeighborsClassifier(n_neighbors= 5, metric= 'minkowski', p = 2)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [439]:
y_pred = classifier.predict(X_test)

print(y_test)
print(y_pred)

[1 1 0 ... 0 0 0]
[1 1 0 ... 0 1 0]


In [440]:
conf = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)

print(conf)
print("Accuracy:", "{:0.5f}".format(accuracy))

[[681 306]
 [307 682]]
Accuracy: 0.73077
